In [ ]:
import os
input_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/transformed_data/Noisy/test"

# 4. Prepare Model and Load Checkpoint

import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np
from pathlib import Path
import os

# Cambiar cantidad de epoca de los pesos
weights_available = list(range(4000, 84000, 4000))
def get_weights_and_parameters(task, parameters):
    if task == 'Real_Denoising':
        # Si se quiere correr otro modelo se requiere cambiar el path
        weights = [Path(f"/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/model/L1Loss_0_99/net_g_{w}.pth") for w in weights_available]
        parameters['LayerNorm_type'] =  'BiasFree'
    return weights, parameters


# Get model weights and parameters
parameters = {'inp_channels':1, 'out_channels':1, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
task = "Real_Denoising"
weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join("/home", "arthemis", "Documents", "pytorch_env", "pytorch_env", "transformers_ruben", "basicsr", "models", "archs","restormer_arch.py"))
model = load_arch['Restormer'](**parameters)
model.cuda()

In [ ]:
# 5. Inference

In [ ]:
from matplotlib import pyplot as plt
from numpy import float64
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from collections import defaultdict

#Datos de validacion articulo
# input_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/validation_data_report"
# noisy_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/Uformer/images"
# out_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/model/L1Loss_0_99/images"

#Datos de validacion de entrenamiento
input_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/transformed_data/val/groundtruth"
noisy_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/transformed_data/val/input"
out_dir = "/home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/model/L1Loss_0_99/images"


os.makedirs(out_dir, exist_ok=True)
extensions = ["jpg", "JPG", "png", "PNG", "jpeg", "JPEG", "bmp", "BMP"]
# files = ['0_1024', '5120_19456','5120_19968','5120_20480']
files = ['1536_2560_a', '2560_3584_a']


img_multiple_of = 8

print(f"\n ==> Running {task} with weights {weights}\n ")
metrics = defaultdict(list[dict[str, float | str | float64]])
with torch.no_grad():
    for weight, index in zip(weights, weights_available):
        checkpoint = torch.load(weight)
        model.load_state_dict(checkpoint["params"])
        model.eval()
        for filepath in tqdm(files):
            torch.cuda.ipc_collect()
            torch.cuda.empty_cache()
            
			#Noisy image input
            im_path = "/".join((noisy_dir, f"{filepath}.png"))
            og = cv2.imread(im_path,-1)
            img = cv2.cvtColor(og, cv2.COLOR_BGR2GRAY)
            img_ = np.reshape(img, (512, 512, 1))

            input_ = torch.from_numpy(img_).float().permute(2, 0, 1).unsqueeze(0).cuda()
            h, w = input_.shape[2], input_.shape[3]
            H, W = (
                ((h + img_multiple_of) // img_multiple_of) * img_multiple_of,
                ((w + img_multiple_of) // img_multiple_of) * img_multiple_of,
            )
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0
            input_ = F.pad(input_, (0, padw, 0, padh), "reflect")
            restored = model(input_)
            restored = restored[:, :, :h, :w]
            restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
            restored = np.squeeze(restored[0])
            
            max_retormer = np.max(restored)
            min_retormer = np.min(restored)            
            
            # Ground truth
            val_im_path = "/".join((input_dir, f"{filepath}.png"))
            val_og = cv2.imread(val_im_path,-1)
            val_img = cv2.cvtColor(val_og, cv2.COLOR_BGR2GRAY)
            
            metrics[f"model_{index}"].append({
				f"file_{filepath}": "",
				"psnr": psnr(val_img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
				"ssim": ssim(val_img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
				"mse": mse(val_img.astype(np.float32), restored)
			})
            filename = os.path.split(input_dir)[-1]
            cv2.imwrite(os.path.join(out_dir, f"{filepath}_{index}.png"), restored)
display(metrics)